In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.textrnn import CuDNNGRULast as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs_sw', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn', 'cudnngrulast_test')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'cudnngrulast_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

VECTOR_DIM = 300
EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem_%d.pkl' % VECTOR_DIM)
RESULT_CSV = os.path.join(RESULTS_PATH, 'cudnngrulast_sw.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.4069 - acc: 0.8695 - val_loss: 0.2101 - val_acc: 0.9382


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.56028, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_0.h5
Epoch 2/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.2167 - acc: 0.9390 - val_loss: 0.1840 - val_acc: 0.9471

Epoch 00002: _f1_monitor improved from 0.56028 to 0.60880, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_0.h5
Epoch 3/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.2050 - acc: 0.9415 - val_loss: 0.1856 - val_acc: 0.9480

Epoch 00003: _f1_monitor improved from 0.60880 to 0.62199, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_0.h5
Epoch 4/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.1971 - acc: 0.9434 - val_loss: 0.1791 - val_acc: 0.9495

Epoch 00004:

105000/105000 [==============================] - 156s 1ms/step - loss: 0.2605 - acc: 0.9175 - val_loss: 0.2702 - val_acc: 0.9143

Epoch 00007: _f1_monitor did not improve from 0.69297
Epoch 8/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.2604 - acc: 0.9177 - val_loss: 0.2619 - val_acc: 0.9177

Epoch 00008: _f1_monitor did not improve from 0.69297
Epoch 9/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.2545 - acc: 0.9198 - val_loss: 0.2685 - val_acc: 0.9164

Epoch 00009: _f1_monitor improved from 0.69297 to 0.69613, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_2.h5

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 10/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.2401 - acc: 0.9233 - val_loss: 0.2633 - val_acc: 0.9179

Epoch 00010: _f1_monitor improved from 0.69613 to 0.69979, saving


Epoch 00008: _f1_monitor improved from 0.78415 to 0.78615, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_4.h5
Epoch 9/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.3682 - acc: 0.8674 - val_loss: 0.3843 - val_acc: 0.8583

Epoch 00009: _f1_monitor did not improve from 0.78615
Epoch 10/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.3609 - acc: 0.8701 - val_loss: 0.3810 - val_acc: 0.8578

Epoch 00010: _f1_monitor improved from 0.78615 to 0.78696, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_4.h5

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 11/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.3437 - acc: 0.8762 - val_loss: 0.3815 - val_acc: 0.8611

Epoch 00011: _f1_monitor did not improve from 0.78696
Epoch

...

Epoch 8/100
105000/105000 [==============================] - 153s 1ms/step - loss: 0.3812 - acc: 0.8676 - val_loss: 0.3879 - val_acc: 0.8637

Epoch 00008: _f1_monitor did not improve from 0.69898
Epoch 9/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.3769 - acc: 0.8693 - val_loss: 0.3934 - val_acc: 0.8617

Epoch 00009: _f1_monitor improved from 0.69898 to 0.70388, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_10.h5

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 10/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.3588 - acc: 0.8756 - val_loss: 0.3909 - val_acc: 0.8617

Epoch 00010: _f1_monitor did not improve from 0.70388
Epoch 11/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.3540 - acc: 0.8767 - val_loss: 0.3954 - val_acc: 0.8601

Epoch 00011: _f1_monitor did not improve from 0.703

Epoch 12/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.4177 - acc: 0.8489 - val_loss: 0.4574 - val_acc: 0.8363

Epoch 00012: _f1_monitor improved from 0.74214 to 0.74370, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_12.h5

Epoch 00012: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 13/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.4140 - acc: 0.8498 - val_loss: 0.4563 - val_acc: 0.8363

Epoch 00013: _f1_monitor did not improve from 0.74370
Epoch 14/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.4118 - acc: 0.8509 - val_loss: 0.4582 - val_acc: 0.8367

Epoch 00014: _f1_monitor improved from 0.74370 to 0.74464, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_12.h5
Epoch 15/100
105000/105000 [=============================

Epoch 9/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.5568 - acc: 0.7957 - val_loss: 0.5862 - val_acc: 0.7839

Epoch 00009: _f1_monitor did not improve from 0.68968
Epoch 10/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.5464 - acc: 0.8021 - val_loss: 0.5871 - val_acc: 0.7813

Epoch 00010: _f1_monitor did not improve from 0.68968
Epoch 11/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.5379 - acc: 0.8056 - val_loss: 0.6095 - val_acc: 0.7738

Epoch 00011: _f1_monitor did not improve from 0.68968
Epoch 12/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.5293 - acc: 0.8092 - val_loss: 0.6083 - val_acc: 0.7788

Epoch 00012: _f1_monitor did not improve from 0.68968

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 13/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.5064 - acc: 0.8170 - val_loss: 0.6077 - val_a

Epoch 11/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.4797 - acc: 0.8276 - val_loss: 0.5474 - val_acc: 0.8039

Epoch 00011: _f1_monitor did not improve from 0.56514

Epoch 00011: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 16th grain: f1_score - 0.5651360390965904, acc - 0.8023333333333333
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.5115 - acc: 0.8328 - val_loss: 0.3329 - val_acc: 0.8964

Epoch 00001: _f1_monitor improved from -inf to 0.68879, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_17.h5
Epoch 2/100
105000/105000 [==============================] - 154s 1ms/step - loss: 0.3411 - acc: 0.8928 - val_loss: 0.3170 - val_acc: 0.9033

Epoch 00002: _f1_monitor improved from 0.68879 to 0.71064, saving model to /root/mounted/projects/ai_challenger_sentiment/data/s


Epoch 00004: _f1_monitor improved from 0.69418 to 0.69715, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_19.h5
Epoch 5/100
105000/105000 [==============================] - 155s 1ms/step - loss: 0.3585 - acc: 0.8708 - val_loss: 0.3433 - val_acc: 0.8742

Epoch 00005: _f1_monitor did not improve from 0.69715
Epoch 6/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.3506 - acc: 0.8736 - val_loss: 0.3444 - val_acc: 0.8738

Epoch 00006: _f1_monitor did not improve from 0.69715
Epoch 7/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.3463 - acc: 0.8760 - val_loss: 0.3414 - val_acc: 0.8755

Epoch 00007: _f1_monitor improved from 0.69715 to 0.70033, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast_test/bicudnngrulast_19.h5
Epoch 8/100
105000/105000 [==============================] - 156s 1ms/step - loss: 0.34

In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6404138941139179
The 1th f1: 0.5333128676198191
The 2th f1: 0.6997916864264417
The 3th f1: 0.6482011909531693
The 4th f1: 0.78894165824573
The 5th f1: 0.7404190999524484
The 6th f1: 0.7325916323449209
The 7th f1: 0.7649953706180431
The 8th f1: 0.6968887644181634
The 9th f1: 0.6734153530367272
The 10th f1: 0.7038795332257072
The 11th f1: 0.7460014304811884
The 12th f1: 0.7446445531266699
The 13th f1: 0.7346137501994644
The 14th f1: 0.6896814930547026
The 15th f1: 0.7152635951564921
The 16th f1: 0.5651360390965904
The 17th f1: 0.7308200299171975
The 18th f1: 0.5759069684360523
The 19th f1: 0.7016094859812103
The average f1 of val is 0.6913264198202328


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)

In [12]:
print("Hello world")

Hello world
